In [1]:
import jieba
import jieba.analyse
import os
import datetime
import pandas as pd
import tushare as ts
import gensim
import re
import numpy as np

ImportError: No module named gensim

# 更新昨日新闻

In [115]:
df = ts.get_latest_news(show_content=True)

line 369: htmlParseEntityRef: expecting ';'


In [6]:
df = pd.read_csv("./news_latest.csv")

In [7]:
df.tail()

,classify,title,time,url,content
1845,美股,纳斯达克拟在新加坡推出中国商品期货合约,10-10 15:37,http://finance.sina.com.cn/stock/usstock/c/201...,此举将为投资者提供直接投资中国船运、铁矿石及原油市场的渠道，同时也有助于扩大纳斯达克在亚洲的...
1846,美股,自助还是求援：危机德银的救赎路,10-10 15:31,http://finance.sina.com.cn/stock/usstock/c/201...,近日来，德意志银行（Deutsche Bank，下称“德银”）深陷困境，其经营健康问题牵动着...
1847,美股,卢森堡首相贝泰尔：英国鱼和熊掌都想要,10-10 15:14,http://finance.sina.com.cn/stock/usstock/c/201...,新浪美股讯 北京时间10日 彭博报道，卢森堡首相贝泰尔表示，欧盟各国必须保持强硬，拒绝在英国...
1848,国内财经,中国降杠杆债转股 专家：救活不救死,10-10 15:08,http://finance.sina.com.cn/china/hgjj/2016-10-...,中国“降杠杆”债转股 专家指“救活不救死”\n中新社北京10月10日电 （记者 赵建华）中国...
1849,国内财经,人民币汇率中间价跌破6.7 央行或不再死守,10-10 15:06,http://finance.sina.com.cn/roll/2016-10-10/doc...,人民币跌破6.7 央行或不再死守\n华夏时报 冉学东\n10月10日，央行确定美元兑人民币汇...


In [4]:
old_df = pd.read_csv("./news.csv")

In [5]:
old_df.head()

,classify,content,date,time,title,url
0,证券,本文为【港股挖掘机】原创文章，欢迎转载，转载务必注明出处。智通财经网新版上线，更多港股及海外...,NaN,10-10 10:20,硬脱欧将给高盛等投行怎样的“硬伤害”?,http://cj.sina.com.cn/article/detail/583552473...
1,证券,本文为【港股挖掘机】原创文章，欢迎转载，转载务必注明出处。智通财经网新版上线，更多港股及海外...,NaN,10-10 10:20,金价大跌未必是坏事？战略性买入机会将出现,http://cj.sina.com.cn/article/detail/583552473...
2,证券,本文为【港股挖掘机】原创文章，欢迎转载，转载务必注明出处。智通财经网新版上线，更多港股及海外...,NaN,10-10 10:20,中粮肉食为IPO甩累赘？猪价见顶回落,http://cj.sina.com.cn/article/detail/583552473...
3,证券,本文为【港股挖掘机】原创文章，欢迎转载，转载务必注明出处。智通财经网新版上线，更多港股及海外...,NaN,10-10 10:20,最严网约车意见稿发布？神州租车偷着乐,http://cj.sina.com.cn/article/detail/583552473...
4,证券,本文为【港股挖掘机】原创文章，欢迎转载，转载务必注明出处。智通财经网新版上线，更多港股及海外...,NaN,10-10 10:20,阿里巴巴影业与斯皮尔伯格的Amblin Partner…,http://cj.sina.com.cn/article/detail/583552473...


In [131]:
del old_df[old_df.columns[0]]

In [133]:
del old_df[old_df.columns[0]]

In [8]:
df = pd.concat([df, old_df])

In [9]:
df.to_csv("news.csv", encoding="utf-8", index=False)

# 手动check数据是否更新，然后用news_update.csv覆盖原始数据news.csv

In [124]:
!cp news_update.csv news.csv

In [10]:
df = pd.read_csv("news.csv")

In [11]:
df.shape

(5805, 6)

In [12]:
df["date"] = df["time"].map(lambda x: datetime.datetime.strptime("2016-" + x.split(' ')[0], "%Y-%m-%d"))

In [14]:
df.head()

,classify,content,date,time,title,url
0,港股,恒生指数低开68点，报23164点；国企指数低开33点，报9567点。,2016-10-17,10-17 01:28,恒指低开68点 报23164点,http://finance.sina.com.cn/stock/hkstock/marke...
1,证券,新浪财经讯 周一早盘，沪深两市双双低开，截止发稿，沪指报3064.69点，涨幅0.03%，深...,2016-10-17,10-17 01:27,快讯：沪深两市小幅高开 沪指涨0.03%,http://finance.sina.com.cn/stock/jsy/2016-10-1...
2,港股,国企债务违约频发，据Wind资讯的统计，自2014年超日债违约以来，内地信用债市场债券逾期金...,2016-10-17,10-17 01:27,债券违约飙 债转股时间换空间,http://finance.sina.com.cn/stock/hkstock/marke...
3,证券,华安证券\n上周大盘是一根小阳线，收盘指数还站上了5周和10周均线。初看起来，大盘似乎又开始...,2016-10-17,10-17 01:27,市场属性已经改变 真正的机会在明年初,http://finance.sina.com.cn/stock/jsy/2016-10-1...
4,证券,据英国金融时报，中化集团或与中国化工的整合协商是由中化董事长宁高宁推动的。,2016-10-17,10-17 01:27,外媒：宁高宁推动中化集团与中国化工整合协商,http://finance.sina.com.cn/stock/s/2016-10-17/...


In [15]:
df[df["classify"] == "国内财经"].shape

(1166, 6)

In [59]:
for word in df["classify"].unique():
    print word

证券
国内财经
期货
美股
生活
nan


# 读取文本

In [19]:
with open("./stop_words_zh.txt", 'r') as f:
    stop_words = set([line.strip() for line in f.readlines()])

In [20]:
now = datetime.datetime.now()

In [21]:
filter_day = now - datetime.timedelta(1)
df[df["classify"] == "国内财经"]["date"] >= datetime.date(2016, 9, 25)

12       True
31       True
33       True
50       True
51       True
59       True
65       True
102      True
103      True
104      True
105      True
106      True
107      True
108      True
111      True
113      True
115      True
118      True
119      True
120      True
124      True
131      True
133      True
136      True
137      True
138      True
139      True
149      True
154      True
158      True
        ...  
5735    False
5736    False
5737    False
5738    False
5739    False
5740    False
5741    False
5742    False
5743    False
5744    False
5745    False
5746    False
5750    False
5751    False
5754    False
5756    False
5757    False
5762    False
5766    False
5772    False
5774    False
5775    False
5779    False
5780    False
5781    False
5782    False
5785    False
5794    False
5799    False
5804    False
Name: date, dtype: bool

In [22]:
def get_word_count(days, topK=20):
    filter_day = now - datetime.timedelta(days)
#     filter_df = df[(df["date"] >= filter_day)]
    filter_df = df[(df["date"] >= datetime.date(filter_day.year, filter_day.month, filter_day.day)) & (df["classify"] == "国内财经")]
    print filter_day, filter_df.shape
    f = open("./document/text", 'w')
    count = {}
    for i in filter_df.index:
        if not isinstance(df.content[i], str):
            continue
        tags = jieba.analyse.textrank(df["content"][i], withWeight=True, topK=topK, allowPOS=('ns', 'n', 'vn', 'v'))
        doc = " ".join([word.encode("utf-8") for word in jieba.cut(filter_df["content"][i]) if not word.encode('utf-8') in stop_words])
        f.write(doc + "\n")
        for word, weight in tags:
            if not count.has_key(word):
                count[word] = 0
            count[word] += weight
    f.close()
    return count

In [144]:
count = get_word_count(1)

2016-10-09 09:59:21.227407 (9, 6)


In [13]:
sort_count = sorted(count.iteritems(), key=lambda x: x[1], reverse=True)

In [14]:
for word, v in sort_count[:100]:
    print word, v

重庆 1.0
铁路 0.488167764729
物流 0.423557448245
内陆 0.334995453248
经济 0.308306393708
开放 0.271885798376
发展 0.256619105416
保税 0.25156489303
中心 0.236179663951
国际 0.22081096412
新区 0.209951827422
两江 0.205361341102
项目 0.203464272079
构建 0.191279938984
联运 0.184132910833
新欧 0.175321584337
体系 0.172920712749
大通道 0.169503170175
依托 0.169067082684
打造 0.166260598818


In [15]:
with open("./filter_words.txt", 'r') as f:
    filter_words = [line.strip().decode('utf-8') for line in f.readlines()]

In [16]:
for word, cnt in sort_count[:100]:
    if word in filter_words:
        continue
    print word, cnt

重庆 1.0
铁路 0.488167764729
物流 0.423557448245
内陆 0.334995453248
开放 0.271885798376
保税 0.25156489303
中心 0.236179663951
国际 0.22081096412
新区 0.209951827422
两江 0.205361341102
项目 0.203464272079
构建 0.191279938984
联运 0.184132910833
新欧 0.175321584337
体系 0.172920712749
大通道 0.169503170175
依托 0.169067082684
打造 0.166260598818


In [17]:
!python word2vec.py ./document/text model vector

2016-08-12 08:33:58,331: INFO: running word2vec.py ./document/text model vector
2016-08-12 08:33:58,334: INFO: collecting all words and their counts
2016-08-12 08:33:58,334: INFO: PROGRESS: at sentence #0, processed 0 words and 0 word types
2016-08-12 08:33:58,336: INFO: collected 706 word types from a corpus of 1382 words and 43 sentences
2016-08-12 08:33:58,336: INFO: total 48 word types after removing those with count<5
2016-08-12 08:33:58,336: INFO: constructing a huffman tree from 48 words
2016-08-12 08:33:58,337: INFO: built huffman tree with maximum node depth 7
2016-08-12 08:33:58,337: INFO: resetting layer weights
2016-08-12 08:33:58,339: INFO: training model with 4 workers on 48 vocabulary and 400 features, using 'skipgram'=1 'hierarchical softmax'=1 'subsample'=0 and 'negative sampling'=0
2016-08-12 08:33:58,341: INFO: reached the end of input; waiting to finish 1 outstanding jobs
2016-08-12 08:33:58,344: INFO: training on 432 words took 0.0s, 99470 words/s
2016-08-12 08:33:

In [18]:
model = gensim.models.Word2Vec.load("model")
result = model.most_similar("债券".decode("utf-8"), topn=20)
for x in result:
    print x[0], x[1]

KeyError: u"word '\u503a\u5238' not in vocabulary"

In [19]:
print sort_count[6][0]
sort_count[6][0] in model.vocab

发展


False

In [23]:
def run_all():
    days = [1, 3, 7, 15]
    with open("./filter_words.txt", 'r') as f:
        filter_words = [line.strip().decode('utf-8') for line in f.readlines()]
    for day in days:
        print("runing %s day" % str(day))
        count = get_word_count(day)
        sort_count = sorted(count.iteritems(), key=lambda x: x[1], reverse=True)
        os.system("python word2vec.py ./document/text model vector")
        model = gensim.models.Word2Vec.load("model")
        f = open("./key_words/key_words_%s-day.txt" % str(day), 'w')
        for word, weight in sort_count[:600]:
            if (word in filter_words) or (not word in model.vocab):
                continue
            f.write("%s:\t" % (word.encode('utf-8')))
#             print word
            similar_words = model.most_similar(word, topn=10)
            for x, v in similar_words:
                f.write("%s " % x.encode("utf-8"))
            f.write("\r\n")
        f.close()

In [24]:
run_all()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ml/92hdr1717w35htxpmg_q2c140000gn/T/jieba.cache
Loading model cost 0.523 seconds.


runing 1 day
2016-10-16 10:05:32.503294 (59, 6)


Prefix dict has been built succesfully.


runing 3 day
2016-10-14 10:05:32.503294 (158, 6)
runing 7 day
2016-10-10 10:05:32.503294 (413, 6)
runing 15 day
2016-10-02 10:05:32.503294 (608, 6)


# 僵尸企业示例

In [23]:
content = df[df["url"] == "http://finance.sina.com.cn/roll/2016-07-29/doc-ifxunyya2641046.shtml"]["content"].iloc[0]

In [24]:
tags = jieba.analyse.textrank(content, withWeight=True, topK=20, allowPOS=('ns', 'n', 'vn', 'v'))

In [25]:
print content

首份僵尸企业研究报告： 国有、集体企业占比高
7月27日，中国人民大学国家发展与战略研究院发布了国内第一份全面研究僵尸企业的报告《中国僵尸企业研究报告——现状、原因和对策》（以下简称《报告》）。
《报告》的主要作者之一、人大国发院副院长聂辉华介绍，如果一个企业在当年和前一年获得的贷款利息率低于正常的市场最低利息率，那么该企业在当年就是僵尸企业。
《报告》根据1998-2013年中国工业企业数据库（包含大约80万家企业和360万个观测值）和1998-2015年上市公司数据库，测算了中国工业部门的僵尸企业比例。结果显示，目前全国工业部门中僵尸企业数量约占工业企业总数的7.51%。
国务院发展研究中心产业经济研究部部长赵昌文建议，从财政、舆论、社会维稳、司法处置、再就业培训、金融监管等方面，全方位集中支持僵尸企业清理所涉及的员工安置和债务处置问题，在全社会就业创造能力仍然较强，银行资本充足率和盈利能力仍然较高之前，尽快清理僵尸企业。
国有、集体企业占比高
《报告》描述了中国工业部门僵尸企业的全貌。
从年份来看，2000-2013年，中国工业部门的僵尸企业比例最高时（2000年）大约30%，此后呈下降趋势，并在2004年之后保持稳定。2005-2013年的工业部门僵尸企业比例大约为7.51%，说明中国僵尸企业的问题是在逐渐缓解的。
从行业来看，利用2013年中国上市公司数据，发现僵尸企业比例最高的五个行业是：钢铁（51.43%）、房地产（44.53%）、建筑装饰（31.76%）、商业贸易（28.89%）和综合类（21.95%）；僵尸企业比例最低的五个行业是：银行（0.00%）、传媒（4.12%）、非银金融（4.65%）、计算机（5.23%）和休闲服务（5.88%）。
从地区来看，经济发展水平较高的东部南部地区僵尸企业比例比较低，而经济发展水平较低的西南、西北和东北地区僵尸企业比例较高。
其比例的分布在2000-2004年和2005-2013年这两个时间段内差别比较大。中部和西南地区的僵尸企业比例在下降，而西部地区的僵尸企业比例有所上升。
从所有制来看，国有和集体企业中僵尸企业的比例最高，民营企业和港澳台及外商企业中僵尸企业的比例相近，且远低于国有和集体企业中僵尸企业的比例。
对此，赵昌文解释，僵尸企业主要是国有类僵尸企业，除产业类国有企业外，还包括大量的地方政府投融资平台公

In [26]:
for word, weight in tags:
    print word, weight

企业 1.0
僵尸 0.681152587671
比例 0.234280126056
报告 0.200040159573
政府 0.172397213677
银行 0.148091457585
中国 0.143238228694
地方 0.129736191519
工业部门 0.107024381549
员工 0.101629730689
社会 0.0990820274336
建议 0.0890861442477
处置 0.0890203164068
重组 0.0862754627321
问题 0.0844619671581
发展 0.0769483632108
过剩 0.0740211665207
登记 0.0726014104068
加快 0.0699666155982
推进 0.0676225550723


# 异常词

In [25]:
def read_words(filepath):
    with open(filepath, 'r') as f:
        words = [(line.split(':')[0], line.split(':')[1].split(' ')) for line in f.readlines()]
    return words

In [26]:
keyw_1d = read_words("key_words/key_words_1-day.txt")
keyw_1d_dict = {k: idx + 1 for idx, (k, v) in enumerate(keyw_1d)}
keyw_3d = read_words("key_words/key_words_3-day.txt")
keyw_3d_dict = {k: idx + 1 for idx, (k, v) in enumerate(keyw_3d)}
keyw_7d = read_words("key_words/key_words_7-day.txt")
keyw_7d_dict = {k: idx + 1 for idx, (k, v) in enumerate(keyw_7d)}
keyw_15d = read_words("key_words/key_words_15-day.txt")
keyw_15d_dict = {k: idx + 1 for idx, (k, v) in enumerate(keyw_15d)}

In [27]:
for (key, value) in keyw_1d[:20]:
    print key, ": ", " ".join(value)

政策 :  	明显 计划 节奏 持有 合同 2017 资本 成交量 出让金 去年同期 

增长 :  	预计 中原 公布 万亿元 我国 M2 进出口 90 接受 情形 

改革 :  	结构性 供给 领域 作用 环境 员工 混合 反弹 创新 改善 

资金 :  	成本 合计 变化 占 收购 投资者 保证金 鲜菜 购房者 人士 

上涨 :  	PPI 环比 百分点 CPI 比上 扩大 小幅 转正 以来 收窄 

城市 :  	导致 撬动 观望 吴昊 土地价格 倍 控制 要求 开发 达到 

楼市 :  	调控 限制 债权 股市 配资 魏桥 成立 集中 杠杆 内地 

土地 :  	股权 配资 股市 伴随 地方 债权 置业 下滑 入市 出让金 

月份 :  	同比 显示 由负 释放 全国 接受 发现 股东 GDP 预期 

房价 :  	出口 能否 一名 结束 发达国家 进口 加强 此轮 走势 看到 

项目 :  	落地 环节 推出 PPP 公司债 外部 判断 回购 普遍 成立 

贷款 :  	股份 财政 原本 合作 近日 下发 延续 房 套 客户 

预期 :  	生活 货币 消息 时 中长期 背后 中介 院长 过剩 当前 

工业 :  	进出口 呈现 反弹 中原 世纪 马拉松 百分点 分别 越来越 M2 

银行 :  	房地产 大量 美国 打算 整个 达到 健康 人士 最终 之间 

首付 :  	李克强 套房 显著 确定 去化 国家统计局 周边 安全 PMI 同策 

融资 :  	一场 优先 普遍 流失 多元化 二线 债权 低于 低位 购房 

调控 :  	楼市 出台 成立 政策 魏桥 完成 环节 内地 节奏 成交量 

影响 :  	因素 回暖 资源配置 数据 也就是说 之下 产生 保持稳定 政策 未来 

数据 :  	资金来源 发力 增速 回升 货币政策 所有制 三季度 之下 因素 往往 



In [29]:
for idx, (word, value) in enumerate(keyw_15d):
    if word in keyw_1d_dict:
        print "1 day", word, keyw_1d_dict[word]
    if word in keyw_3d_dict:
        print "3 days", word, keyw_3d_dict[word]
    if word in keyw_7d_dict:
        print "7 days", word, keyw_7d_dict[word]
    if word in keyw_15d_dict:
        print "15 days", word, keyw_15d_dict[word]
    print "\n"

1 day 政策 1
3 days 政策 1
7 days 政策 1
15 days 政策 1


1 day 城市 6
3 days 城市 4
7 days 城市 2
15 days 城市 2


1 day 住房 40
3 days 住房 51
7 days 住房 29
15 days 住房 3


1 day 增长 2
3 days 增长 2
7 days 增长 3
15 days 增长 4


1 day 人民币 32
3 days 人民币 42
7 days 人民币 4
15 days 人民币 5


1 day 房价 10
3 days 房价 12
7 days 房价 12
15 days 房价 6


1 day 楼市 7
3 days 楼市 14
7 days 楼市 8
15 days 楼市 7


1 day 调控 18
3 days 调控 21
7 days 调控 9
15 days 调控 8


1 day 改革 3
3 days 改革 8
7 days 改革 5
15 days 改革 9


1 day 项目 11
3 days 项目 23
7 days 项目 6
15 days 项目 10


1 day 上涨 5
3 days 上涨 3
7 days 上涨 13
15 days 上涨 11


1 day 限购 57
3 days 限购 110
7 days 限购 66
15 days 限购 12


1 day 北京 23
3 days 北京 43
7 days 北京 17
15 days 北京 13


1 day 资金 4
3 days 资金 15
7 days 资金 7
15 days 资金 14


1 day 贷款 12
3 days 贷款 7
7 days 贷款 20
15 days 贷款 15


1 day 银行 15
3 days 银行 16
7 days 银行 14
15 days 银行 16


1 day 政府 39
3 days 政府 20
7 days 政府 11
15 days 政府 17


1 day 国家 43
3 days 国家 9
7 days 国家 10
15 days 国家 18


1 day 土地 8
3 days 土地 37
7 days 土地 21
15 days 土地 19


1 

In [29]:
word = "廊坊"
print keyw_1d_dict[word], keyw_3d_dict[word], keyw_7d_dict[word], keyw_15d_dict[word]

KeyError: '\xe5\xbb\x8a\xe5\x9d\x8a'

In [392]:
word = "试验区"
print keyw_1d_dict[word], keyw_3d_dict[word], keyw_7d_dict[word], keyw_15d_dict[word]

9 39 88 150


In [393]:
word = "上海"
print keyw_1d_dict[word], keyw_3d_dict[word], keyw_7d_dict[word], keyw_15d_dict[word]

39 77 60 65


In [395]:
word = "监管"
print keyw_1d_dict[word], keyw_3d_dict[word], keyw_7d_dict[word], keyw_15d_dict[word]

12 49 72 118


In [441]:
for word in keyw_1d_dict:
    if word in keyw_3d_dict:
        if keyw_3d_dict[word] - keyw_1d_dict[word] < -30:
            if (word in keyw_7d_dict) and (word in keyw_15d_dict):
                print word, keyw_15d_dict[word], keyw_7d_dict[word], keyw_3d_dict[word], keyw_1d_dict[word]

 猪肉 331 153 181 242
城市 4 3 1 200
质量 330 243 191 228
贷款 45 37 27 120
回落 116 62 51 319
蔬菜 534 329 222 341
利率 23 38 45 163
工作 12 27 54 109
中心 237 275 153 291
需求 78 57 48 107
杠杆 160 167 127 158
证券 214 188 172 306
情况 31 49 67 317
月份 48 20 10 160
促进 117 179 202 290
工业 34 35 120 245
导致 213 165 186 293
资本 41 89 185 349
国际 49 91 61 108
发行 111 77 104 284
开展 54 42 40 126
反弹 470 327 226 318
油价 219 259 141 198
业务 53 50 42 176
投放 96 94 100 147
平台 46 30 59 310
推进 14 34 50 113
继续 30 11 12 253
行业 10 9 15 159
基金 60 59 74 127
力度 323 228 123 314
金融机构 110 143 78 180
降低 134 274 167 211
下滑 316 315 223 261
需要 106 83 150 210
研究 131 119 91 199
预计 173 117 62 187
人口 158 126 81 118
流动性 74 58 53 104
融合 156 264 147 193
经济学家 367 211 125 267
合作 161 136 126 208
政策 3 8 9 51
服务业 137 71 32 287
民间 20 25 24 101
指数 73 15 7 213
操作 24 24 31 153
数据 33 18 14 58
涨幅 226 291 163 337
有限公司 153 116 152 254
债券 76 79 68 265
显示 108 44 23 91
土地 43 36 55 139
保持 71 84 64 264
下跌 249 194 121 325
走势 526 498 288 326
建设 17 33 65 248
下行 220 174 2

In [431]:
word_list = ["改革", "泡沫", "抑制", "负债", "债券", "流动性"]

In [650]:
def run_all_word_list(word_list):
    days = [1, 3, 7, 15]
    for day in days:
        print("runing %s day" % str(day))
        count = get_word_count(day)
        sort_count = sorted(count.iteritems(), key=lambda x: x[1], reverse=True)
        os.system("python word2vec.py ./document/text model vector")
        model = gensim.models.Word2Vec.load("model")
        f = open("./key_words/related_words_%s-day.txt" % str(day), 'w')
        for word in word_list:
            word = word.decode("utf-8")
            if (not word in model.vocab):
                continue
            f.write("%s:\t" % (word.encode('utf-8')))
#             print word
            similar_words = model.most_similar(word, topn=20)
            for x, v in similar_words:
                f.write("%s " % x.encode("utf-8"))
            f.write("\r\n")
        f.close()

In [651]:
run_all_word_list(word_list)

runing 1 day
runing 3 day
runing 7 day
runing 15 day


# 获取每日关键词

In [444]:
def get_everyday_words(date, topK=20):
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    filter_df = df[(df["date"] == date) & (df["classify"] == "国内财经")]
    count = {}
    for i in filter_df.index:
        if not isinstance(df.content[i], str):
            continue
        tags = jieba.analyse.textrank(df["content"][i], withWeight=True, topK=topK, allowPOS=('ns', 'n', 'vn', 'v'))
        doc = " ".join([word.encode("utf-8") for word in jieba.cut(filter_df["content"][i]) if not word.encode('utf-8') in stop_words])
        for word, weight in tags:
            if not count.has_key(word):
                count[word] = 0
            count[word] += weight
    return count

In [446]:
count = get_everyday_words("2016-08-03")

In [447]:
sort_count = sorted(count.iteritems(), key=lambda x: x[1], reverse=True)

In [448]:
for word, value in sort_count[:100]:
    print word, value

企业 15.5350493056
经济 11.7390431032
中国 8.20743940387
投资 5.94851504787
增长 5.51118794889
改革 5.09312203116
创新 4.78312902729
市场 4.61786925587
人民币 4.25217764602
重组 3.78539323153
贸易 3.45570174097
增速 3.44465129114
发展 3.25008223982
试点 3.24707653045
试验区 3.24017828072
资金 2.8378985614
产能 2.66815818849
监管 2.64404134468
煤炭 2.38466412018
金融 2.19273128883
货币 2.04705554341
收入 1.78771179263
中央 1.77035888568
政府 1.76667113078
资产 1.71351184804
表示 1.69299208736
广东 1.6509303861
融资 1.63882279681
实施 1.63595963935
消费 1.57046857917
部门 1.51605551919
推广 1.47585345912
推出 1.47378375067
记者 1.44057494772
问题 1.37201890696
方面 1.36776924969
交易 1.35385855526
成本 1.35216605153
公司 1.31006464666
预期 1.30747636233
实现 1.30691949439
产业 1.24513379611
转移 1.22270483881
加工 1.18435746918
结构性 1.18113604509
销售 1.17654841053
管理 1.16248040972
上海 1.14525434845
汇率 1.14219600266
出现 1.1385458789
支持 1.09900177363
供给 1.09469362287
出口 1.07417196722
集团 1.0305197156
价格 1.0
就业 1.0
原油 1.0
居住 1.0
收费站 1.0
项目 1.0
提高 0.99772342739
政策 0.996082593938
过剩 0.

In [452]:
start_date = datetime.date(2016, 7, 12)
end_date = datetime.date.today()

In [474]:
current = start_date
words_weight_change = {}
while current != end_date:
    date = "%s-%s-%s" % (current.year, current.month, current.day)
    print(date)
    count = get_everyday_words(date, 50)
    sort_count = sorted(count.iteritems(), key=lambda x: x[1], reverse=True)
    for idx, (word, value) in enumerate(sort_count):
        if not words_weight_change.has_key(word):
            words_weight_change[word] = []
        words_weight_change[word].append((date, idx + 1, value))
    current = current + datetime.timedelta(1)

2016-7-12
2016-7-13
2016-7-14
2016-7-15
2016-7-16
2016-7-17
2016-7-18
2016-7-19
2016-7-20
2016-7-21
2016-7-22
2016-7-23
2016-7-24
2016-7-25
2016-7-26
2016-7-27
2016-7-28
2016-7-29
2016-7-30
2016-7-31
2016-8-1
2016-8-2


In [499]:
words_weight_change[u"僵尸"]

[('2016-7-14', 823, 0.2108231821686243),
 ('2016-7-16', 410, 0.2947455916538843),
 ('2016-7-18', 445, 0.440194989175863),
 ('2016-7-19', 863, 0.23769019943859993),
 ('2016-7-20', 1283, 0.09507866341589448),
 ('2016-7-22', 997, 0.17431392859282263),
 ('2016-7-25', 240, 0.8707114912513908),
 ('2016-7-26', 310, 0.4876844370414466),
 ('2016-7-28', 56, 2.605977116236191),
 ('2016-7-29', 150, 0.6811525876705348),
 ('2016-7-30', 510, 0.15057128472069203),
 ('2016-8-1', 532, 0.2606068727280894),
 ('2016-8-2', 244, 0.8318093422257817)]

In [511]:
words_weight_change[u"调研"]

[('2016-7-13', 1263, 0.07461977204272985),
 ('2016-7-14', 88, 1.9333676289150383),
 ('2016-7-15', 213, 1.1657799613305686),
 ('2016-7-17', 320, 0.2618956429579662),
 ('2016-7-20', 830, 0.21449729048216273),
 ('2016-7-21', 950, 0.1774270085594601),
 ('2016-7-25', 367, 0.5696376439475217),
 ('2016-7-26', 354, 0.4332925518852302),
 ('2016-7-27', 766, 0.18755289479687337),
 ('2016-7-29', 904, 0.07298042597150056),
 ('2016-8-1', 835, 0.14771191462144692)]

In [507]:
def cold_word(word_date_count):
    prev_index = len(word_date_count) / 5
    weights = [weight for (date, rank, weight) in word_date_count]
    prev_weight = np.mean(weights[:-prev_index])
    current_weight = np.mean(weights[-prev_index:])
    if (prev_weight - current_weight) / prev_weight > 0.8:
        return True
    else:
        return False

In [508]:
len(words_weight_change)

6612

In [510]:
for word in words_weight_change:
    if cold_word(words_weight_change[word]):
        print word

机场
调研
区分
热点问题
上榜
纳税人
上海市
能力
进出口
反向
回答
铁路
着力
全市
签署
外贸
收费
学院
税负
核准
允许
论坛
污染
公示
磋商
干预
衔接
毕业生


/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
